This file is a compilation of all the codes to show the flow of our analysis process. However, they might not work as it was not tested to run due to the high computational time it requires. The output and run files are named as per analysis in the folder.

# Importing Libraries

In [ ]:
!pip install wordcloud
!pip install nltk
!pip install text2emotion

nltk.download('punkt')
nltk.download('words')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')

In [ ]:
import pandas as pd 
import numpy as np 
import time

#For Text Cleaning
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

#For EDA
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

#For text vectorizing
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora
from gensim import models
import random

# Topic Modelling
import gensim
import pickle 
import os 
os.environ.update({'MALLET_HOME':r'C:/mallet-2.0.8/'})
mallet_path = r'C:\\mallet-2.0.8\\bin\\mallet'

# Emotion Analysis 
# import text2emotion as te

# pos tagging 
from collections import Counter

# ploting 
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# VADER generation 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# ER
import sys
import json
import ast

# Classification 

from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn import model_selection, svm, feature_extraction, model_selection, manifold, preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K



# Helper Functions
## Preprocessing

In [ ]:
def preprocess(review):
    review = " ".join([stemmer.stem(w.lower()) for w in word_tokenize(re.sub('[^a-zA-Z]+', ' ', review.replace("<br />", ""))) if not w in stop_words])
    return review
    
def preprocessG(review):
    review = " ".join([stemmer.stem(w.lower()) for w in word_tokenize(review) if not w in stop_words])
    return review

def getSmallerCorpus(news):
    X = news.loc[:, news.columns != 'label']
    y = news['label']
    new_corpus, unwanted_data, new_corpus_y, unwanted_data_y = train_test_split(X,y,train_size= 0.45,  random_state=0, stratify=y)
    new_corpus['label'] = new_corpus_y
    return new_corpus


## Topic Modelling

In [ ]:
def docs2vecs(docs, dictionary):
    # docs is a list of documents returned by corpus2docs.
    # dictionary is a gensim.corpora.Dictionary object.
    vecs1 = [dictionary.doc2bow(doc) for doc in docs]
    return vecs1

def save_model(classifier):
    save_classifier = open("topicModel14.pickle","wb") #binary write
    pickle.dump(classifier, save_classifier)
    save_classifier.close()

def format_topics_sentences(ldamodel, corpus, data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        # row = sorted(row, key=lambda x: (x[1]))
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        sent_topics_df = sent_topics_df.append(pd.Series([round(row[0][1],4),round(row[1][1],4),round(row[2][1],4),round(row[3][1],4),round(row[4][1],4),round(row[5][1],4),round(row[6][1],4),round(row[7][1],4),round(row[8][1],4),round(row[9][1],4),round(row[10][1],4),round(row[11][1],4),round(row[12][1],4),round(row[13][1],4)]), ignore_index=True)
        print(i)

    sent_topics_df.columns = ['Topic 0','Topic 1','Topic 2','Topic 3','Topic 4','Topic 5', 'Topic 6','Topic 7','Topic 8','Topic 9','Topic 10','Topic 11','Topic 12','Topic 13']
       
    # Add original text to the end of the output
    data_sentence = [' '.join(w) for w in data]
    contents = pd.Series(data_sentence)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
    

## Name Entity Recognition

In [ ]:
def getEntityDic(df):
  entity_dic = {}
  counter = 1
  total_records = df.shape[0]
  for index, row in df.iterrows():
    text = row['text'] + " "+ row['title']
    sys.stdout.write('\rCompletion progress: ' + str(counter) + ' of ' + str(total_records) + " articles")
    sys.stdout.flush()

    ## Put full stop behind each sentence
    new = []
    if "\n" in text:
        text = text.replace(" \n", ". ")
    text_list = text.split(". ")
    for x in text_list:
        new.append(x + ".")
    
    ## Cycle through each sentence
    for x in new:
        sent_tokens = word_tokenize(x)
        tagged_sent = nltk.pos_tag(sent_tokens)
        ne_tree = nltk.ne_chunk(tagged_sent)
        ne_list = extract_ne_from_tree(ne_tree)
        
        ## Insert into dictionary
        for y in ne_list:
            if y[0] not in entity_dic:
                entity_dic[y[0]] = {}
            if y[0] in entity_dic:
                string = y[1].lower()
                if string not in entity_dic[y[0]]:
                    entity_dic[y[0]][string] = 0
                if string in entity_dic[y[0]]:
                    entity_dic[y[0]][string] += 1
    counter += 1
  return entity_dic

def extract_ne_from_tree ( tree ):
    result = []
    for s in tree.subtrees():
        label = s.label()
        if (label == 'PERSON' or label == 'ORGANIZATION' or label == 'GPE'):
            leaves = s.leaves()
            ne = ''
            for l in leaves:
                ne = ne + ' ' + l[0]
            result.append((label, ne[1:]))
    return result

def addEntitiesToDataframe2(df):

    entity_dic = {}
    counter = 1
    total_records = df.shape[0]
    i = 0
    for index, row in df.iterrows():
        entity_dic['PERSON'] = {}
        entity_dic['ORGANIZATION'] = {}
        entity_dic['GPE'] = {}
        text = row['text'] + " "+ row['title']
        sys.stdout.write('\rCompletion progress: ' + str(counter) + ' of ' + str(total_records) + " articles")
        sys.stdout.flush()

        ## Put full stop behind each sentence
        new = []
        if "\n" in text:
          text = text.replace(" \n", ". ")
        text_list = text.split(". ")
        for x in text_list:
            new.append(x + ".")

        ## Cycle through each sentence
        for x in new:
            sent_tokens = word_tokenize(x)
            tagged_sent = nltk.pos_tag(sent_tokens)
            ne_tree = nltk.ne_chunk(tagged_sent)
            ne_list = extract_ne_from_tree(ne_tree)

            ## Insert into dictionary
            for y in ne_list:
                if y[0] not in entity_dic:
                    entity_dic[y[0]] = {}
                if y[0] in entity_dic:
                    string = y[1].lower()
                    if string not in entity_dic[y[0]]:
                        entity_dic[y[0]][string] = 0
                    if string in entity_dic[y[0]]:
                        entity_dic[y[0]][string] += 1
        counter += 1

        # Get df index
        ind = index_list[i]

        # get summation values
        total_p = 0
        for x in entity_dic['PERSON']:
          total_p += entity_dic['PERSON'][x]

        total_o = 0
        for x in entity_dic['ORGANIZATION']:
          total_o += entity_dic['ORGANIZATION'][x]

        total_g = 0
        for x in entity_dic['GPE']:
          total_g += entity_dic['GPE'][x]
        
        # insert into DF
        df.loc[ind:ind+1, 'person entities'] = str(entity_dic['PERSON'])
        df.loc[ind:ind+1, 'organisation entities'] = str(entity_dic['ORGANIZATION'])
        df.loc[ind:ind+1, 'location entities'] = str(entity_dic['GPE'])
        df.loc[ind:ind+1, 'person entity count'] = total_p
        df.loc[ind:ind+1, 'organisation entity count'] = total_o
        df.loc[ind:ind+1, 'location entity count'] = total_g
        entity_dic = {}
        i+=1
        
    return df

def attachTopEntities(df, fakeTop, realTop):
    store = []
    for x in trueTop:
        for y in trueTop[x]:
        store.append(("(" + str(x) + ")" + y))

    for x in fakeTop:
        for y in fakeTop[x]:
        item = ("(" + str(x) + ")" + y)
        if item not in store:
            store.append(item)

    for x in store:
        df[x] = 0

    return df, store

def getTopEntityDic(entity_dic, n):
    print()
    bigstore = {}
    top10entities = {}
    for x in entity_dic:
        first = str(x)[0] + "_"
        bigstore[x] = []
        print(x)
        print("----------------")
        sorted_dic = sorted(entity_dic[x], key=entity_dic[x].get, reverse=True)
        count = 0
        for y in sorted_dic:
        top10entities[y] = entity_dic[x][y]
        print(y.ljust(40), ":" + str(entity_dic[x][y]))
        count += 1
            if count == n:
                break
        top10 = dict(sorted(top10entities.items(), key=lambda item: item[1], reverse = True))
        count = 0
        result = []
        for z in top10:
        count += 1
        result.append(str(z))
            if count == n:
                bigstore[x] = result 
                break
        print()
    return bigstore

def getFinalEntityDF(finalEntitydf, store):
    total_records = finalEntitydf.shape[0]
    for index, row in finalEntitydf.iterrows():
        sys.stdout.write('\rCompletion progress: ' + str(index+1) + ' of ' + str(total_records) + " articles")
        sys.stdout.flush()
        # Person Entities
        json_string = row['person entities']
        dicp = ast.literal_eval(json_string)
        for x in dicp:
        item = "(PERSON)" + str(x)
            if item in store:
                finalEntitydf.loc[index, item] = dicp[x]

        # Location Entities
        json_string = row['location entities']
        dicp = ast.literal_eval(json_string)
        for x in dicp:
        item = "(GPE)" + str(x)
            if item in store:
                finalEntitydf.loc[index, item] = dicp[x]
            
        # GPE Entities
        json_string = row['organisation entities']
        dicp = ast.literal_eval(json_string)
        for x in dicp:
        item = "(ORGANISATION)" + str(x)
            if item in store:
                finalEntitydf.loc[index, item] = dicp[x]
    return finalEntitydf

## POS tagging

In [ ]:
def generate_pos_tag_dist(df):
    
    """Takes in a df : this dataframe contains the emotions,
        and outputs the dataframe with additional pos tags above that have no more than 30% missing """
        
    emotion_text = list(df['text'])
    counts = []
    for sentences in emotion_text:
        tokens = nltk.word_tokenize(sentences)    
        tags = nltk.pos_tag(tokens)
        counts.append(Counter( tag for word,  tag in tags))

    # creating the pos count feature and setting non rare pos tag features ,
    df_post_dist = pd.DataFrame.from_records(counts)
    df_post_dist_non_null = df_post_dist.loc[:,df_post_dist.columns[df_post_dist.isnull().mean() < 0.7]].reset_index()
    df_post_dist_non_null.fillna(0,inplace=True)
    
    # combining the dataframe 
    return df_post_dist_non_null


def group_pos_features(pos_tagged_df):

    """This function takes in the POS feature dataframe and group POS tags together.
      Comments means that the POS tags in this data set is not prevelant as they either are not present or contain too much missing values

      Unpresent POS tags  for now it is known to be insignificant as mentioned in the research paper :
          pos_tagged_df['group_e'] = pos_tagged_df['EX']
          pos_tagged_df['NR'] + pos_tagged_df['NPS']
          pos_tagged_df['group_p'] = pos_tagged_df['PDT'] + pos_tagged_df['POS'] + pos_tagged_df['PP']
          pos_tagged_df['group_t'] = pos_tagged_df['TO']
          pos_tagged_df['group_u'] = pos_tagged_df['UH'] 
          pos_tagged_df['group_w'] = pos_tagged_df['WDT'] + pos_tagged_df['WP'] + pos_tagged_df['WP$'] + pos_tagged_df['WRB']
          # Could be a future work to include using different POS tagging dictionaries
     """ 
    #pos_group = ['group_c','group_d','group_f','group_i','group_j','group_m','group_n','group_r','group_v']
    pos_list = ['CC','CD','DT','FW','IN','JJ','JJR','JJS','MD','NN', 'NNS', 'RBR','RB','VB','VBD', 'VBN','VBP','VBZ']
    delete_pos_list = []
    pos_dict = {}
    for pos_l in pos_list:
      
      if pos_l in pos_tagged_df.columns:
        pos_dict[pos_l] = np.array(list(pos_tagged_df[pos_l]))
        delete_pos_list.append(pos_l)
      else:
        pos_dict[pos_l] = list(zerolistmaker(len(pos_tagged_df)))
      
    #print(np.sum([pos_dict['CC'],pos_dict['CD']] , axis=0))
    pos_tagged_df['group_c'] = np.sum([pos_dict['CC'],pos_dict['CD']], axis=0) 
    pos_tagged_df['group_d'] = pos_dict['DT'] 
    pos_tagged_df['group_f'] = pos_dict['FW'] 
    pos_tagged_df['group_i'] = pos_dict['IN']
    pos_tagged_df['group_j'] = np.sum([pos_dict['JJ'],pos_dict['JJR'], pos_dict['JJS']], axis=0)  
    
    pos_tagged_df['group_m'] = pos_dict['MD']
    pos_tagged_df['group_n'] = np.sum([pos_dict['NN'],pos_dict['NNS']], axis=0) 
    
    # changed RBP to RBR 
    pos_tagged_df['group_r'] = np.sum([pos_dict['RBR'],pos_dict['RB']], axis=0) 
    pos_tagged_df['group_v'] =  np.sum([pos_dict['VB'],pos_dict['VBD'], pos_dict['VBN'],  pos_dict['VBP'],  pos_dict['VBZ']], axis=0)  

    return pos_tagged_df, delete_pos_list



## Emotions/Sentiment 

In [ ]:
#emotion
def emotion_detection(sents):
    """Main algo for convertion for the 5 emotions """
    sent_emotion = te.get_emotion(sents)
    return sent_emotion
    

# Generating the Emotions 
def generate_emotions(news_dup):
    
    """Use to generate the dataframe that appends the orginal text and the emotion label vector"""
    
    emotion_list = []
    for i, row in news_dup.iterrows():
        emotion_dict = emotion_detection(row[2])
        emotion_dict['text'] = row[2]
        emotion_dict['label'] = row[3]
        emotion_list.append(emotion_dict)
        
        
    emotion_df = pd.DataFrame(emotion_list)
    horizontal_stack = news_dup.merge(emotion_df, how='left', on='text')
    horizontal_stack.drop(['index'], inplace=True, axis=1)       
            
    return horizontal_stack
        
# sentiment
# generating Vader https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664
def vader_score_generation(emo_pos_df):
    
    """Generates the vader scores for the dataframe to create neu pos neg tags base on the text """
    sid = SentimentIntensityAnalyzer()
    polarity_Score = emo_pos_df['text'].apply(lambda review: sid.polarity_scores(review))
    df_p_scores = polarity_Score.apply(pd.Series).reset_index() # generating the scores and appending it the dataframe end 
    emo_pos_dist_pscore =  emo_pos_df.merge(df_p_scores, left_on='index', right_on='index')
    return emo_pos_dist_pscore

def zerolistmaker(n):
    listofzeros = [0] * n
    return listofzeros

## Classification

In [ ]:
def randomsearchmethod(clf, param_dist, n_iter_search):

    # Utility function to report best scores
    def report(results, n_top=3):
        for i in range(1, n_top + 1):
            candidates = np.flatnonzero(results['rank_test_score'] == i)
            for candidate in candidates:
                print("Model with rank: {0}".format(i))
                print("Mean validation score: {0:.3f} (std: {1:.3f})"
                      .format(results['mean_test_score'][candidate],
                              results['std_test_score'][candidate]))
                print("Parameters: {0}".format(results['params'][candidate]))
                print("")

    random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                       n_iter=n_iter_search)

    random_search.fit(Train_X_Tfidf,Train_Y)
    report(random_search.cv_results_)
    
## code attention layer
def attention_layer(inputs, neurons):
    x = layers.Permute((2,1))(inputs)
    x = layers.Dense(neurons, activation="softmax")(x)
    x = layers.Permute((2,1), name="attention")(x)
    x = layers.multiply([inputs, x])
    return x

    
def makeROC(classifiers, title, rocX_test, rocy_test):
    # Define a result table as a DataFrame
    result_table = pd.DataFrame(columns=['classifiers', 'fpr','tpr','auc'])

    # Train the models and record the results
    for cls in classifiers:
        model = cls
        yproba = model.predict_proba(rocX_test)[::,1]

        fpr, tpr, _ = roc_curve(rocy_test,  yproba)
        auc = roc_auc_score(rocy_test, yproba)

        result_table = result_table.append({'classifiers':cls.__class__.__name__,
                                            'fpr':fpr, 
                                            'tpr':tpr, 
                                            'auc':auc}, ignore_index=True)

    # Set name of the classifiers as index labels
    result_table.set_index('classifiers', inplace=True)
    fig = plt.figure(figsize=(8,6))

    for i in result_table.index:
        plt.plot(result_table.loc[i]['fpr'], 
                 result_table.loc[i]['tpr'], 
                 label="{}, AUC={:.3f}".format(i, result_table.loc[i]['auc']))

    plt.plot([0,1], [0,1], color='orange', linestyle='--')

    plt.xticks(np.arange(0.0, 1.1, step=0.1))
    plt.xlabel("False Positive Rate", fontsize=15)

    plt.yticks(np.arange(0.0, 1.1, step=0.1))
    plt.ylabel("True Positive Rate", fontsize=15)

    plt.title(('ROC Curve Analysis '+title), fontweight='bold', fontsize=15)
    plt.legend(prop={'size':13}, loc='lower right')

    plt.show()
    

# Data Preprocessing
## Standardizing format

In [ ]:
DIR = 'sample_data/'
SUBDIR = 'original/'

df1truth = pd.read_csv(DIR + SUBDIR + "True.csv") 
df1truth["label"] = "REAL"
#First 3 words of the text column indicate the branch of the news company(reuter) that is publishing the news article
#Remove the first 3 words of all text in the text column 
newtext = []
for i in df1truth['text']:
    newtext.append(' '.join(i.split(' ')[3:]))
df1truth['text'] = newtext

df1fake = pd.read_csv(DIR + SUBDIR + "Fake.csv") 
df1fake["label"] = "FAKE"

df2 = pd.read_csv(DIR + SUBDIR  + "fake_or_real_news.csv")

df3 = pd.read_csv(DIR + SUBDIR + "news_articles.csv") 
df3 = df3.replace(["Real"], "REAL")
df3 = df3.replace(["Fake"], "FAKE")

#this dataframe is used to create CompiledNewsData.xlsx
dfcompiled = pd.concat([df1truth[["title","text","label","subject"]],df1fake[["title","text","label","subject"]],df2[["title","text","label"]],df3[["title","text","label","type"]]])
dfcompiled = dfcompiled[dfcompiled['text'].notna()]
maindf = dfcompiled[["title","text","label"]]
maindfG = maindf.copy()

## Data Cleaning (without grammar)

In [ ]:
#For Creating Cleaned dataframe without grammar

stop_words = set(stop)
stemmer = PorterStemmer()

#The bottom two lines will remove all stop words and stem the column text, takes very long do not run
maindf['text'] = maindf.apply(lambda x: preprocess(x['text']), axis=1)
maindf['title'] = maindf.apply(lambda x: preprocess(x['title']), axis=1)

#Export the new cleaned data into excel file
maindf.to_excel(DIR + 'CompiledNewsData.xlsx', index = False, header=True)

## Data Cleaning (With Grammar) - For ER

In [ ]:
#The bottom two lines will remove all stop words and stem the column text, takes very long do not run


maindfG['text'] = maindfG.apply(lambda x: preprocessG(x['text']), axis=1)
maindfG['title'] = maindfG.apply(lambda x: preprocessG(x['title']), axis=1)

#Export the new cleaned data into excel file
maindfG.to_excel(DIR + 'CompiledNewsDataWithGrammar.xlsx', index = False, header=True)

## Converting labels and removing duplicates 

In [ ]:
#This is the main compiled dataframe to work with
df = pd.read_excel(DIR + "CompiledNewsData.xlsx")
dfgrammar = pd.read_excel(DIR + "CompiledNewsDataWithGrammar.xlsx")  

#Dataframes had some duplicate titles and text
df = df.drop_duplicates()
df = df.dropna()
df['label'] = df['label'].apply(lambda x: 1 if x =='REAL' else 0)

dfgrammar = dfgrammar.drop_duplicates()
dfgrammar = dfgrammar.dropna()
dfgrammar['label'] = dfgrammar['label'].apply(lambda x: 1 if x =='REAL' else 0)

## Split data into a smaller corpus

In [ ]:
news20 = getSmallerCorpus(df)
news20.to_excel(DIR + 'common_df_20k.xlsx')
news20G = getSmallerCorpus(dfgrammar)
news20G.to_excel(DIR + 'common_df_20k_Grammar.xlsx')

# EDA on whole dataset

## View True False proportion

In [ ]:
news = df.copy()
print (news['label'].value_counts())

Tasks = list(news[["title","label"]].groupby('label').count()["title"])
my_labels = 'Fake','Real'
plt.pie(Tasks,labels=my_labels,autopct='%1.1f%%')
plt.axis('equal')
plt.show()

## Word Cloud for Real News

In [ ]:
#Compile all "labeled real into one text variable"
fullreal = ""
for i in news[news["label"] == 1]["text"]:
    fullreal += (str(i) + " ")
    
# Create and generate a word cloud image:
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(fullreal)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

## Word Cloud for Fake News

In [ ]:
#Compile all "labeled fake into one text variable"
fullfake = ""
for i in news[news["label"] == 0]["text"]:
    fullfake += (str(i) + " ")
    
# Create and generate a word cloud image:
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(fullfake)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# FE - Topic Modelling

In [ ]:
news = df['text']

news_docs = [[w for w in new.split()] for new in news]
news_dictionary = gensim.corpora.Dictionary(news_docs)
news_vecs = docs2vecs(news_docs, news_dictionary)

 ## Finding optimal k value - Coherence Score


In [3]:
# Can take a long time to run. In this case we are going to  k_max=10.
import datetime
print(datetime.datetime.now())

from gensim.models import CoherenceModel

model_list = []
coherence_values = []
model_topics = []

for num_topics in range(2, 20, 2):
    #sg_lda_x = gensim.models.ldamodel.LdaModel(corpus=sg_vecs, id2word=sg_dictionary, num_topics=num_topics)
    sg_lda_x = gensim.models.wrappers.LdaMallet(mallet_path, iterations=100, corpus=news_vecs, num_topics=num_topics, id2word=news_dictionary)
    coherencemodel = CoherenceModel(model=sg_lda_x, texts=news_docs, dictionary=news_dictionary, coherence='c_v')
    model_topics.append(num_topics)
    model_list.append(sg_lda_x)
    coherence_values.append(coherencemodel.get_coherence())
    print("#Topics: " + str(num_topics) + " Score: " + str(coherencemodel.get_coherence()))

print(datetime.datetime.now())

# Show graph

import matplotlib.pyplot as plt

limit=20; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

## View Top 10 words from 8 Topics

In [ ]:
no_topics=8

# mallet_path = r"C://mallet-2.0.8/bin/mallet" # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=news_vecs, num_topics=no_topics, id2word=news_dictionary)

vecTop = ldamallet.show_topics()
topics = []
for i in range(0, no_topics):
    top10values = vecTop[i][1]
    # print (top10values)
    topics.append([val.split('*')[1] for val in top10values.split('+')])

for t in topics:
    print (t)

## View Top 10 words from 14 Topics

In [ ]:
no_topics=14

# mallet_path = r"C://mallet-2.0.8/bin/mallet" # update this path

# save classifier
ldamallet14 = gensim.models.wrappers.LdaMallet(mallet_path, corpus=news_vecs, num_topics=no_topics, id2word=news_dictionary)
save_model(ldamallet14)
    
classifier_saved = open("topicModel14.pickle", "rb") #binary read
classifier_load = pickle.load(classifier_saved)
classifier_saved.close()

vecTop14 = classifier_load.show_topics(14)
topics14 = []
for i in range(0, no_topics):
    top10values14 = vecTop14[i][1]
    topics14.append([val.split('*')[1] for val in top10values14.split('+')])

    
for t in topics14:
    print (t)

## Retreive Topic Distribution for each news

In [ ]:
#Find most dominant topic

import numpy as np
import pandas as pd

classifier_saved = open("topicModel14.pickle", "rb") #binary read
classifier_load = pickle.load(classifier_saved)
classifier_saved.close()
df_topic_sents_keywords = format_topics_sentences(ldamodel=classifier_load, corpus=news_vecs, data=news_docs)

# Format
# df_dominant_topic = df_topic_sents_keywords.reset_index()
df_topic_sents_keywords.columns = ['Topic 0','Topic 1','Topic 2','Topic 3','Topic 4','Topic 5', 'Topic 6','Topic 7','Topic 8','Topic 9','Topic 10','Topic 11','Topic 12','Topic 13','Text']

# Show
df_topic_sents_keywords.head(10)
df_topic_sents_keywords.to_excel('topic_distribution_14.xlsx')

# FE - Name Entity Recognition

## Fake News Entities and Aggregated Domain Counts

In [ ]:
df1fake = news20G[news20G['label'] == 0]                                 # 1) Get fake news article dataframe
index_list = df1fake.index.tolist()                                 # 2) Get index of fake dataframe
# fakeEntityDic = getEntityDic(df1fake)                               # 3) Get sorted dict of {fake entities : count}
# fakeTop = getTopEntityDic(fakeEntityDic, 10)                        # 4) Get dic of {entitiy type : [sorted list of top 10 entities]}
df1_fakeEntities = addEntitiesToDataframe2(df1fake)     

## View Top Entities for Fake News

In [ ]:
# df1fake = pd.read_csv("Fake.csv", engine= 'python') 
# df1fake["label"] = "FAKE"
# df1_fakeEntities = addEntitiesToDataframe(df1fake)
# df1_fakeEntities.to_csv("df1_fakeEntities.csv", index = False)
# print (df1_fakeEntities.head(3))

# df_fake = df1fake['text'].to_frame()
# fake_entity_dic = getEntityDic(df_fake)
# print()
# print("======================================================")
# getTopEntites(fake_entity_dic, 15)
# with open('fake_entity.json', 'w') as fp:
#     json.dump(fake_entity_dic, fp)

## Real News Entities and Aggregated Domain Counts

In [ ]:
# Real News
df1true = news20G[news20G['label'] == 1]                                 # 1) Get real news article dataframe
index_list = df1true.index.tolist()                                 # 2) Get index of real dataframe
# trueEntityDic = getEntityDic(df1true)                               # 3) Get sorted dict of {real entities : count}
# trueTop = getTopEntityDic(trueEntityDic, 10)                        # 4) Get dic of {entitiy type : [sorted list of top 10 entities]}
df1_trueEntities = addEntitiesToDataframe2(df1true)                 # 5) Get DF of entities, entity count per article

## View Top Entities for Fake News

In [ ]:
#  df1_trueEntities = addEntitiesToDataframe(df1truth)
#  df1_trueEntities.to_csv("df1_trueEntities.csv", index = False)
#  df1_trueEntities.head(3)

# # Cycle through true articles
# df_true = df1truth['text'].to_frame()
# true_entity_dic = getEntityDic(df_true)
# print()
# print("======================================================")
# getTopEntites(true_entity_dic, 15)
# with open('true_entity_dic.json', 'w') as fp:
#     json.dump(true_entity_dic, fp)

## Combining into a DataFrame

In [ ]:
# Combining Real and Fake News Dataframes
finalEntitydf = pd.concat([df1_trueEntities, df1_fakeEntities])     # Union fake & real dataframes
finalEntitydf.sort_index(ascending=True, inplace=True)              # Re-index them to original state

# Attaching Top Entities as Column Headers to Main Dataframe                   
finalEntitydf, store = attachTopEntities(finalEntitydf, fakeTop, trueTop)       

# Get Final DF with Top Entity counts
mainEntitydf = getFinalEntityDF(finalEntitydf, store)

del mainEntitydf['person entities']
del mainEntitydf['organisation entities']
del mainEntitydf['location entities']
del emotional_df_stack_groupped_pos['Unnamed: 0']

# POS Tag, Emotions/Sentiment Analysis

## Generate Emotion Distribution

In [ ]:
emp_df = generate_emotions(news20)
emp_df.to_excel('emotion_analysis_final.xlsx')
emp_df = pd.read_excel('emotion_analysis_final.xlsx')

# Generating the POS tag dist 
emo_pos_df = generate_pos_tag_dist(emp_df)
emp_df = emp_df.reset_index()
emp_df = emp_df[emp_df.columns.drop(list(emp_df.filter(regex='index')))]
emp_df = emp_df.rename(columns={emp_df.columns[0]: 'index'})
comb_emo_pos_dist =  emp_df.merge(emo_pos_df, left_on='index', right_on='index')

# genrating vader text 
emo_pos_dist_pscore = vader_score_generation(comb_emo_pos_dist)
emo_pos_dist_pscore.to_excel('emo_pos_dist_pscore.xlsx')

# Generating of Pos Grouping Features 
emotional_df_stack = pd.read_excel('emo_pos_dist_pscore.xlsx')
del emotional_df_stack['compound']

emotional_df_stack_groupped_pos, pos_tags = group_pos_features(emotional_df_stack)
emotional_df_stack_groupped_pos = emotional_df_stack_groupped_pos.loc[:, (emotional_df_stack_groupped_pos != 0).any(axis=0)]
emotional_df_stack_groupped_pos.to_excel('pos_groupings_vader_emotion.xlsx')


## Visualization

### PCA

In [ ]:
# PCA 
emotional_df_stack = pd.read_excel('pos_groupings_vader_emotion.xlsx')
news_pca = emotional_df_stack.iloc[:, 5:]

pca = PCA(n_components=2)
news_pca_components = pca.fit_transform(news_pca)
principalDf = pd.DataFrame(data = news_pca_components, columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, news_pca[['label']]], axis = 1) 

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = [1,0]
colors = ['r', 'g']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['label'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

### Covariance Matrix (Sentiments and Emotions)

In [ ]:
# Printing Corrvariance value 
corr_sentiment_emotion = pd.read_excel('sentiment_emotion_corr.xlsx')
c = corr_sentiment_emotion.corr().abs()
s = c.unstack()
so = s.sort_values(kind="quicksort")
print(so[:-14])

# Covariance heatmap 
cov_mat = corr_sentiment_emotion.corr()
# plot the heatmap
sn.heatmap(cov_mat, 
        xticklabels=cov_mat.columns,
        yticklabels=cov_mat.columns)

### Overall Distribution

In [ ]:
#Emotions
plotting_df_emotion_senti_pso = emotional_df_stack.iloc[:, 4:]
plotting_df_emotion_senti_pso.groupby('label').mean().reset_index().plot(x="label", y=["Happy", "Angry", "Surprise", "Sad", "Fear"], kind="bar")

In [ ]:
# POS tag
plotting_df_emotion_senti_pso.groupby('label').mean().reset_index().plot(x="label", y=list(pos_tags), kind="bar")

In [ ]:
# POS tag groupings
pos_groups_dist = emotional_df_stack.columns[-9:]
emotional_df_stack.groupby('label').mean().reset_index().plot(x="label", y=pos_groups_dist, kind="bar")

In [ ]:
# Sentiments
vader_tags = ['neg','neu','pos']
plotting_df_emotion_senti_pso.groupby('label').mean().reset_index().plot(x="label", y=vader_tags, kind="bar")

In [ ]:
# output for classification
emotional_df_stack.drop(columns=pos_tags, inplace=True)
emotional_df_stack = emotional_df_stack.loc[:, (emotional_df_stack != 0).any(axis=0)]
emotional_df_stack.to_excel('pos_groupings_vader_emotion_final.xlsx')

horizontal_stack = emotional_df_stack.merge(mainEntitydf, how='left', on='index')
horizontal_stack = horizontal_stack.loc[:,~horizontal_stack.columns.duplicated()]

# uncomment if needed
# del horizontal_stack['label']
# del horizontal_stack['label_x']
# del horizontal_stack['title_y']
# del horizontal_stack['text_y']
# del horizontal_stack[',']
# del horizontal_stack['.']
horizontal_stack.to_excel('final_feature_engineered.xlsx')

# Classification

## Train Test Split

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['text'],df['label'],test_size=0.3)

## Generate Word Vector

In [ ]:
fulllist = []
for i in df['text']:
    fulllist.append(i)

#Set maxfeatures to 5000 maybe next time0
Tfidf_vect = TfidfVectorizer(max_features=None)
Tfidf_vect.fit(fulllist)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

## Stochastic Gradient Descent

In [ ]:
SGD = SGDClassifier(loss='log', penalty='l2', alpha=1e-3, n_iter_no_change=5)

param_dist = {
    'penalty': ['elasticnet','l2'],
    'alpha': [10 ** x for x in range(-6, 1)],
    'l1_ratio': [0, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1],
}

n_iter_search = 20

randomsearchmethod(SGD,param_dist, n_iter_search)

## ADA Boost

In [ ]:
ADA = AdaBoostClassifier()

param_dist = {
    'learning_rate' : [0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1],
    'n_estimators': [100,200,300,400],
    'base_estimator': [DecisionTreeClassifier(random_state=0, max_depth=1), DecisionTreeClassifier(random_state=0, max_depth=2),DecisionTreeClassifier(random_state=0, max_depth=3)]
}

n_iter_search = 3

randomsearchmethod(ADA,param_dist, n_iter_search)

## Random Forest

In [ ]:
RF = RandomForestClassifier()

param_dist = {
    'max_depth': [5,10,15,20,25],
    'n_estimators': [70,80,85,90,100]
}

n_iter_search = 20

randomsearchmethod(RF,param_dist, n_iter_search)

## XGBoost

In [ ]:
XGB = XGBClassifier()

param_dist = {
    'max_depth': [0,5,10,15,20,25],
    'n_estimators': [50, 100, 150, 200, 250],
    'reg_alpha': [0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1]
}

n_iter_search = 3

randomsearchmethod(XGB,param_dist, n_iter_search)

## LSTM

## Word2Vec Model

In [ ]:
#nlp = gensim_api.load("word2vec-google-news-300")
## create list of lists of unigrams
corpus = Train_X

lst_corpus = []
for string in corpus:
   lst_words = string.split()
   lst_grams = [" ".join(lst_words[i:i+1]) 
               for i in range(0, len(lst_words), 1)]
   lst_corpus.append(lst_grams)

bigrams_detector = gensim.models.phrases.Phrases(lst_corpus, delimiter=" ".encode(), min_count=5, threshold=10)
bigrams_detector = gensim.models.phrases.Phraser(bigrams_detector)

nlp = gensim.models.word2vec.Word2Vec(lst_corpus, size=300, window=8, min_count=1, sg=1, iter=30)

## tokenize text
tokenizer = kprocessing.text.Tokenizer(lower=True, split=' ', 
                     oov_token="NaN", 
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(lst_corpus)
dic_vocabulary = tokenizer.word_index
## create sequence
lst_text2seq= tokenizer.texts_to_sequences(lst_corpus)
## padding sequence
X_train = kprocessing.sequence.pad_sequences(lst_text2seq, 
                    maxlen=15, padding="post", truncating="post")

corpus = Test_X

## create list of n-grams
lst_corpus = []
for string in corpus:
    lst_words = string.split()
    lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, 
                 len(lst_words), 1)]
    lst_corpus.append(lst_grams)
    
## detect common bigrams using the fitted detectors
lst_corpus = list(bigrams_detector[lst_corpus])
## text to sequence with the fitted tokenizer
lst_text2seq = tokenizer.texts_to_sequences(lst_corpus)

## padding sequence
X_test = kprocessing.sequence.pad_sequences(lst_text2seq, maxlen=15,
             padding="post", truncating="post")

## start the matrix (length of vocabulary x vector size) with all 0s
embeddings = np.zeros((len(dic_vocabulary)+1, 300))
for word,idx in dic_vocabulary.items():
    ## update the row with vector
    try:
        embeddings[idx] =  nlp[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

### Declare NN Layers

In [ ]:

## input
x_in = layers.Input(shape=(15,))
## embedding
x = layers.Embedding(input_dim=embeddings.shape[0],  
                     output_dim=embeddings.shape[1], 
                     weights=[embeddings],
                     input_length=15, trainable=False)(x_in)
## apply attention
x = attention_layer(x, neurons=15)
## 2 layers of bidirectional lstm
x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.3, 
                         return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.3))(x)
## final dense layers
x = layers.Dense(32, activation='relu')(x)
y_out = layers.Dense(2, activation='softmax')(x)
## compile
NNmodel = models.Model(x_in, y_out)
NNmodel.compile(loss=tensorflow.keras.losses.SparseCategoricalCrossentropy(),
              optimizer='adam', metrics=['accuracy'])

NNmodel.summary()

### Traning the Model

In [ ]:
## encode y
dic_y_mapping = {n:label for n,label in 
                 enumerate(np.unique(Train_Y))}
inverse_dic = {v:k for k,v in dic_y_mapping.items()}
y_train = np.array([inverse_dic[y] for y in Train_Y])
## train
training = NNmodel.fit(x=X_train, y=y_train, batch_size=256, epochs=2, shuffle=True, verbose=0, validation_split=0.2)
## plot loss and accuracy
metrics = [k for k in training.history.keys() if ("loss" not in k) and ("val" not in k)]
fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True)
ax[0].set(title="Training")
ax11 = ax[0].twinx()
ax[0].plot(training.history['loss'], color='black')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss', color='black')
for metric in metrics:
    ax11.plot(training.history[metric], label=metric)
ax11.set_ylabel("Score", color='steelblue')
ax11.legend()
ax[1].set(title="Validation")
ax22 = ax[1].twinx()
ax[1].plot(training.history['val_loss'], color='black')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss', color='black')
for metric in metrics:
     ax22.plot(training.history['val_'+metric], label=metric)
ax22.set_ylabel("Score", color="steelblue")
plt.show()

## Prediction
predicted_prob = NNmodel.predict(X_test)
predicted = [dic_y_mapping[np.argmax(pred)] for pred in 
             predicted_prob]

## Accuracy

In [ ]:
# replace ngram_range = (1,2) for bigram
NB = MultinomialNB().fit(Train_X_Tfidf,Train_Y)

SGD = SGDClassifier(loss='log', penalty= 'l2', l1_ratio= 0.05, alpha= 1e-05, n_iter_no_change=5).fit(Train_X_Tfidf,Train_Y)

ADA = AdaBoostClassifier(n_estimators= 300, learning_rate= 0.2, base_estimator= DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,)).fit(Train_X_Tfidf, Train_Y)

RF = RandomForestClassifier(n_estimators=100, max_depth=25, random_state=0).fit(Train_X_Tfidf, Train_Y)

XGB = XGBClassifier(n_estimators = 200, max_depth=15, reg_alpha=0.05).fit(Train_X_Tfidf, Train_Y)

# Save to file in the current working directory
pkl_filename = "BaseTFIDFNB.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(NB, file)

pkl_filename = "BaseTFIDFSGD.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(SGD, file)

pkl_filename = "BaseTFIDFADA.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(ADA, file)

pkl_filename = "BaseTFIDFRF.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(RF, file)

pkl_filename = "BaseTFIDFXGB.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(XGB, file)

# Load from file
pkl_filename = "BaseTFIDFNB.pkl"
with open(pkl_filename, 'rb') as file:
    NB = pickle.load(file)
predictions_NB = NB.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("****NB Report****")
print("MultinomialNB Accuracy:",metrics.accuracy_score(Test_Y, predictions_NB))
print('\nClasification report:\n', classification_report(Test_Y, predictions_NB))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y, predictions_NB)  )
print()
print("*"*36)

# Load from file
pkl_filename = "BaseTFIDFSGD.pkl"
with open(pkl_filename, 'rb') as file:
    SGD = pickle.load(file)
# predict the labels on validation dataset
predictions_SGD = SGD.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("****SGD Report****")
print("SGD Accuracy:",metrics.accuracy_score(Test_Y, predictions_SGD))
print('\nClasification report:\n', classification_report(Test_Y, predictions_SGD))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y, predictions_SGD))
print()
print("*"*36)

# Load from file
pkl_filename = "BaseTFIDFADA.pkl"
with open(pkl_filename, 'rb') as file:
    ADA = pickle.load(file)
predictions_ADA = ADA.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("****ADABoost Report****")
print("AdaBoostClassifier Accuracy:",metrics.accuracy_score(Test_Y, predictions_ADA))
print('\nClasification report:\n', classification_report(Test_Y, predictions_ADA))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y, predictions_ADA) )
print()
print("*"*36)

# Load from file
pkl_filename = "BaseTFIDFRF.pkl"
with open(pkl_filename, 'rb') as file:
    RF = pickle.load(file)
predictions_RF = RF.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("****RF Report****")
print("RandomForest Accuracy:",metrics.accuracy_score(Test_Y, predictions_RF))
print('\nClasification report:\n', classification_report(Test_Y, predictions_RF))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y, predictions_RF)  )
print()
print("*"*36)

# Load from file
pkl_filename = "BaseTFIDFXGB.pkl"
with open(pkl_filename, 'rb') as file:
    XGB = pickle.load(file)
predictions_XGB = XGB.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("****XGB Report****")
print("XGBoost Accuracy:",metrics.accuracy_score(Test_Y, predictions_XGB))
print('\nClasification report:\n', classification_report(Test_Y, predictions_XGB))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y, predictions_XGB)  )

print("****LSTM Report****")
print("LSTM Accuracy:",metrics.accuracy_score(Test_Y, predicted))
print('\nClasification report:\n', classification_report(Test_Y, predicted))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y, predicted)  )

## ROC 

In [ ]:
print()
print("*"*36)

makeROC([NB, XGB, RF, ADA, SGD],"TFIDF Real Vs Fake Performance", Test_X_Tfidf, Test_Y)

## Importing and joining data

In [ ]:
dfemotion = pd.read_excel("sample_data/pos_groupings_vader_emotion.xlsx")
dfemotion = dfemotion.drop(['text','title','Unnamed: 0','index','label_y','compound'], axis=1)
dftopic = pd.read_excel("sample_data/topic_distribution_14.xlsx")
dftopic = dftopic.drop(['Unnamed: 0','Text'], axis=1)

entity = pd.read_csv("sample_data/Entity_final.csv")
entity = entity.drop(['title','text','label','index','person entities','organisation entities','location entities'], axis=1)

#Split text set to 20k rows
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['text'],df['label'],train_size= 0.45, stratify=df['label'], random_state=0)

#TFIDF Vectorize New Sample Set
fulllist = []
for i in Train_X:
    fulllist.append(i)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(fulllist)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)

#Convert to dataframe
Train_X_df = pd.DataFrame(Train_X_Tfidf.toarray(), columns=Tfidf_vect.get_feature_names())
Train_Y = Train_Y.fillna(0)

#Combine dataframes
dftopic = pd.concat([dftopic, Train_X_df], axis=1)
dfemotion = pd.concat([dfemotion, entity[['person entity count', 'organisation entity count' , 'location entity count']]], axis=1)
save_y = Train_Y.copy()

#Create test and train text set
Train_X_Topic, Test_X_Topic, Train_Y_Topic, Test_Y_Topic = model_selection.train_test_split(dftopic,Train_Y, train_size= 0.4, stratify=Train_Y, random_state=0)
Train_X_Emotion, Test_X_Emotion, Train_Y_Emotion, Test_Y_Emotion = model_selection.train_test_split(dfemotion,Train_Y, train_size= 0.4, stratify=Train_Y, random_state=0)
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Train_X_df,Train_Y,train_size= 0.4, stratify=Train_Y, random_state=0)

## Base Model Classification

In [ ]:
# replace ngram_range = (1,2) for bigram
TFIDFNB = MultinomialNB().fit(Train_X,Train_Y)

TFIDFSGD = SGDClassifier(loss='log', penalty= 'l2', l1_ratio= 0.05, alpha= 1e-05, n_iter_no_change=5).fit(Train_X,Train_Y)

TFIDFADA = AdaBoostClassifier(n_estimators= 300, learning_rate= 0.2, base_estimator= DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,)).fit(Train_X,Train_Y)

TFIDFRF = RandomForestClassifier(n_estimators=100, max_depth=25, random_state=0).fit(Train_X,Train_Y)

TFIDFXGB = XGBClassifier(n_estimators = 200, max_depth=15, reg_alpha=0.05).fit(Train_X,Train_Y)

# Save to file in the current working directory
pkl_filename = "BaseExtractTFIDFNB.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(TFIDFNB, file)

pkl_filename = "BaseExtractTFIDFSGD.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(TFIDFSGD, file)

pkl_filename = "BaseExtractTFIDFADA.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(TFIDFADA, file)

pkl_filename = "BaseExtractTFIDFRF.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(TFIDFRF, file)

pkl_filename = "BaseExtractTFIDFXGB.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(TFIDFXGB, file)

# Load from file
pkl_filename = "BaseExtractTFIDFNB.pkl"
with open(pkl_filename, 'rb') as file:
    TFIDFNB = pickle.load(file)

# Load from file
pkl_filename = "BaseExtractTFIDFSGD.pkl"
with open(pkl_filename, 'rb') as file:
    TFIDFSGD = pickle.load(file)

# Load from file
pkl_filename = "BaseExtractTFIDFADA.pkl"
with open(pkl_filename, 'rb') as file:
    TFIDFADA = pickle.load(file)

# Load from file
pkl_filename = "BaseExtractTFIDFRF.pkl"
with open(pkl_filename, 'rb') as file:
    TFIDFRF = pickle.load(file)

# Load from file
pkl_filename = "BaseExtractTFIDFXGB.pkl"
with open(pkl_filename, 'rb') as file:
    TFIDFXGB = pickle.load(file)

# replace ngram_range = (1,2) for bigram
predictions_NB = TFIDFNB.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("****NB Report****")
print("MultinomialNB Accuracy:",metrics.accuracy_score(Test_Y, predictions_NB))
print('\nClasification report:\n', classification_report(Test_Y, predictions_NB))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y, predictions_NB)  )
print()
print("*"*36)


# predict the labels on validation dataset
predictions_SGD = TFIDFSGD.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("****SGD Report****")
print("SGD Accuracy:",metrics.accuracy_score(Test_Y, predictions_SGD))
print('\nClasification report:\n', classification_report(Test_Y, predictions_SGD))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y, predictions_SGD))
print()
print("*"*36)


predictions_ADA = TFIDFADA.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("****ADABoost Report****")
print("AdaBoostClassifier Accuracy:",metrics.accuracy_score(Test_Y, predictions_ADA))
print('\nClasification report:\n', classification_report(Test_Y, predictions_ADA))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y, predictions_ADA) )
print()
print("*"*36)


predictions_RF = TFIDFRF.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("****RF Report****")
print("RandomForest Accuracy:",metrics.accuracy_score(Test_Y, predictions_RF))
print('\nClasification report:\n', classification_report(Test_Y, predictions_RF))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y, predictions_RF)  )
print()
print("*"*36)


predictions_XGB = TFIDFXGB.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("****XGB Report****")
print("XGBoost Accuracy:",metrics.accuracy_score(Test_Y, predictions_XGB))
print('\nClasification report:\n', classification_report(Test_Y, predictions_XGB))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y, predictions_XGB)  )

## Base + Topic Model Classification

In [ ]:
print("*"*36)
print("SGD Optimization")
SGD = SGDClassifier(loss='log', penalty='l2', alpha=1e-3, n_iter_no_change=5)
param_dist = {
    'penalty': ['elasticnet','l2'],
    'alpha': [10 ** x for x in range(-6, 1)],
    'l1_ratio': [0, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1],
}
n_iter_search = 20
randomsearchmethod(SGD,param_dist, n_iter_search, Train_X_Topic,Train_Y_Topic)

print("*"*36)
print("ADABoost Optimization")
ADA = AdaBoostClassifier()
param_dist = {
    'learning_rate' : [0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1],
    'n_estimators': [100,200,300,400],
    'base_estimator': [DecisionTreeClassifier(random_state=0, max_depth=1), DecisionTreeClassifier(random_state=0, max_depth=2),DecisionTreeClassifier(random_state=0, max_depth=3)]
}
n_iter_search = 2
randomsearchmethod(ADA,param_dist, n_iter_search, Train_X_Topic,Train_Y_Topic)

#Restart this one it ran on emotion
print("*"*36)
print("Random Forest Optimization")
RF = RandomForestClassifier()
param_dist = {
    'max_depth': [5,10,15,20,25,30,35,40],
    'n_estimators': [70,80,85,90,100,110,120,130,140]
}
n_iter_search = 20
randomsearchmethod(RF,param_dist, n_iter_search, Train_X_Topic,Train_Y_Topic)

print("*"*36)
print("XGBoost Optimization")
XGB = XGBClassifier()
param_dist = {
    'max_depth': [0,5,10,15,20,25],
    'n_estimators': [50, 100, 150, 200, 250],
    'reg_alpha': [0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1]
}
n_iter_search = 2
randomsearchmethod(XGB,param_dist, n_iter_search, Train_X_Topic,Train_Y_Topic)

# Save Data
TopicADA = AdaBoostClassifier(n_estimators= 350, learning_rate= 0.1, base_estimator= DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,)).fit(Train_X_Topic,Train_Y_Topic)

pkl_filename = "TopicTFIDFADA.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(TopicADA, file)

    # replace ngram_range = (1,2) for bigram
TopicNB = MultinomialNB().fit(Train_X_Topic,Train_Y_Topic)

TopicSGD = SGDClassifier(loss='log', penalty= 'elasticnet', l1_ratio= 0.1, alpha= 0.0001, n_iter_no_change=5).fit(Train_X_Topic,Train_Y_Topic)

TopicADA = AdaBoostClassifier(n_estimators= 350, learning_rate= 0.1, base_estimator= DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,)).fit(Train_X_Topic,Train_Y_Topic)

TopicRF = RandomForestClassifier(n_estimators=140, max_depth=40, random_state=0).fit(Train_X_Topic,Train_Y_Topic)

TopicXGB = XGBClassifier(n_estimators = 120, max_depth=10, reg_alpha=0.8).fit(Train_X_Topic,Train_Y_Topic)

# Save to file in the current working directory
pkl_filename = "TopicTFIDFNB.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(TopicNB, file)

pkl_filename = "TopicTFIDFSGD.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(TopicSGD, file)

pkl_filename = "TopicTFIDFADA.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(TopicADA, file)

pkl_filename = "TopicTFIDFRF.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(TopicRF, file)

pkl_filename = "TopicTFIDFXGB.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(TopicXGB, file)


    # Load from file
pkl_filename = "TopicTFIDFNB.pkl"
with open(pkl_filename, 'rb') as file:
    TopicNB = pickle.load(file)

# Load from file
pkl_filename = "TopicTFIDFSGD.pkl"
with open(pkl_filename, 'rb') as file:
    TopicSGD = pickle.load(file)

# Load from file
pkl_filename = "TopicTFIDFADA.pkl"
with open(pkl_filename, 'rb') as file:
    TopicADA = pickle.load(file)

# Load from file
pkl_filename = "TopicTFIDFRF.pkl"
with open(pkl_filename, 'rb') as file:
    TopicRF = pickle.load(file)

# Load from file
pkl_filename = "TopicTFIDFXGB.pkl"
with open(pkl_filename, 'rb') as file:
    TopicXGB = pickle.load(file)

predictions_NB = TopicNB.predict(Test_X_Topic)
# Use accuracy_score function to get the accuracy
print("****NB Report****")
print("MultinomialNB Accuracy:",metrics.accuracy_score(Test_Y_Topic, predictions_NB))
print('\nClasification report:\n', classification_report(Test_Y_Topic, predictions_NB))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_Topic, predictions_NB)  )
print()
print("*"*36)


# predict the labels on validation dataset
predictions_SGD = TopicSGD.predict(Test_X_Topic)
# Use accuracy_score function to get the accuracy
print("****SGD Report****")
print("SGD Accuracy:",metrics.accuracy_score(Test_Y_Topic, predictions_SGD))
print('\nClasification report:\n', classification_report(Test_Y_Topic, predictions_SGD))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_Topic, predictions_SGD))
print()
print("*"*36)


predictions_ADA = TopicADA.predict(Test_X_Topic)
# Use accuracy_score function to get the accuracy
print("****ADABoost Report****")
print("AdaBoostClassifier Accuracy:",metrics.accuracy_score(Test_Y_Topic, predictions_ADA))
print('\nClasification report:\n', classification_report(Test_Y_Topic, predictions_ADA))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_Topic, predictions_ADA) )
print()
print("*"*36)


predictions_RF = TopicRF.predict(Test_X_Topic)
# Use accuracy_score function to get the accuracy
print("****RF Report****")
print("RandomForest Accuracy:",metrics.accuracy_score(Test_Y_Topic, predictions_RF))
print('\nClasification report:\n', classification_report(Test_Y_Topic, predictions_RF))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_Topic, predictions_RF)  )
print()
print("*"*36)


predictions_XGB = TopicXGB.predict(Test_X_Topic)
# Use accuracy_score function to get the accuracy
print("****XGB Report****")
print("XGBoost Accuracy:",metrics.accuracy_score(Test_Y_Topic, predictions_XGB))
print('\nClasification report:\n', classification_report(Test_Y_Topic, predictions_XGB))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_Topic, predictions_XGB)  )

## ROC 
makeROC([TopicNB, TopicXGB, TopicRF, TopicADA, TopicSGD],"TFIDF + Topic probabilities Real Vs Fake Performance", Test_X_Topic, Test_Y_Topic)

## Generic Model Classification

In [ ]:
print("*"*36)
print("SGD Optimization")
SGD = SGDClassifier(loss='log', penalty='l2', alpha=1e-3, n_iter_no_change=5)
param_dist = {
    'penalty': ['elasticnet','l2'],
    'alpha': [10 ** x for x in range(-6, 1)],
    'l1_ratio': [0, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1],
}
n_iter_search = 20
randomsearchmethod(SGD,param_dist, n_iter_search, Train_X_Emotion,Train_Y_Emotion)

print("*"*36)
print("ADABoost Optimization")
ADA = AdaBoostClassifier()
param_dist = {
    'learning_rate' : [0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1],
    'n_estimators': [100,200,300,400],
    'base_estimator': [DecisionTreeClassifier(random_state=0, max_depth=1), DecisionTreeClassifier(random_state=0, max_depth=2),DecisionTreeClassifier(random_state=0, max_depth=3)]
}
n_iter_search = 20
randomsearchmethod(ADA,param_dist, n_iter_search, Train_X_Emotion,Train_Y_Emotion)

print("*"*36)
print("Random Forest Optimization")
RF = RandomForestClassifier()
param_dist = {
    'max_depth': [5,10,15,20,25,30,35,40],
    'n_estimators': [70,80,85,90,100,110,120,130,140]
}
n_iter_search = 20
randomsearchmethod(RF,param_dist, n_iter_search, Train_X_Emotion,Train_Y_Emotion)

print("*"*36)
print("XGBoost Optimization")
XGB = XGBClassifier()
param_dist = {
    'max_depth': [0,5,10,15,20,25],
    'n_estimators': [50, 100, 150, 200, 250],
    'reg_alpha': [0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1]
}
n_iter_search = 20
randomsearchmethod(XGB,param_dist, n_iter_search, Train_X_Emotion,Train_Y_Emotion)

# replace ngram_range = (1,2) for bigram
EmotionNB = MultinomialNB().fit(Train_X_Emotion,Train_Y_Emotion)

EmotionSGD = SGDClassifier(loss='log', penalty= 'elasticnet', l1_ratio= 0.8, alpha= 1e-05, n_iter_no_change=5).fit(Train_X_Emotion,Train_Y_Emotion)

EmotionADA = AdaBoostClassifier(n_estimators= 400, learning_rate= 0.1, base_estimator= DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,)).fit(Train_X_Emotion,Train_Y_Emotion)

EmotionRF = RandomForestClassifier(n_estimators=130, max_depth=40, random_state=0).fit(Train_X_Emotion,Train_Y_Emotion)

EmotionXGB = XGBClassifier(n_estimators = 250, max_depth=20, reg_alpha=0.05).fit(Train_X_Emotion,Train_Y_Emotion)

# Save to file in the current working directory
pkl_filename = "EmotionNB.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(EmotionNB, file)

pkl_filename = "EmotionSGD.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(EmotionSGD, file)

pkl_filename = "EmotionADA.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(EmotionADA, file)

pkl_filename = "EmotionRF.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(EmotionRF, file)

pkl_filename = "EmotionXGB.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(EmotionXGB, file)

    # Load from file
pkl_filename = "EmotionNB.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionNB = pickle.load(file)

# Load from file
pkl_filename = "EmotionSGD.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionSGD = pickle.load(file)

# Load from file
pkl_filename = "EmotionADA.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionADA = pickle.load(file)

# Load from file
pkl_filename = "EmotionRF.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionRF = pickle.load(file)

# Load from file
pkl_filename = "EmotionXGB.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionXGB = pickle.load(file)

predictions_NB = EmotionNB.predict(Test_X_Emotion)
# Use accuracy_score function to get the accuracy
print("****NB Report****")
print("MultinomialNB Accuracy:",metrics.accuracy_score(Test_Y_Emotion, predictions_NB))
print('\nClasification report:\n', classification_report(Test_Y_Emotion, predictions_NB))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_Emotion, predictions_NB)  )
print()
print("*"*36)


# predict the labels on validation dataset
predictions_SGD = EmotionSGD.predict(Test_X_Emotion)
# Use accuracy_score function to get the accuracy
print("****SGD Report****")
print("SGD Accuracy:",metrics.accuracy_score(Test_Y_Emotion, predictions_SGD))
print('\nClasification report:\n', classification_report(Test_Y_Emotion, predictions_SGD))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_Emotion, predictions_SGD))
print()
print("*"*36)


predictions_ADA = EmotionADA.predict(Test_X_Emotion)
# Use accuracy_score function to get the accuracy
print("****ADABoost Report****")
print("AdaBoostClassifier Accuracy:",metrics.accuracy_score(Test_Y_Emotion, predictions_ADA))
print('\nClasification report:\n', classification_report(Test_Y_Emotion, predictions_ADA))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_Emotion, predictions_ADA) )
print()
print("*"*36)


predictions_RF = EmotionRF.predict(Test_X_Emotion)
# Use accuracy_score function to get the accuracy
print("****RF Report****")
print("RandomForest Accuracy:",metrics.accuracy_score(Test_Y_Emotion, predictions_RF))
print('\nClasification report:\n', classification_report(Test_Y_Emotion, predictions_RF))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_Emotion, predictions_RF)  )
print()
print("*"*36)


predictions_XGB = EmotionXGB.predict(Test_X_Emotion)
# Use accuracy_score function to get the accuracy
print("****XGB Report****")
print("XGBoost Accuracy:",metrics.accuracy_score(Test_Y_Emotion, predictions_XGB))
print('\nClasification report:\n', classification_report(Test_Y_Emotion, predictions_XGB))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_Emotion, predictions_XGB)  )

#ROC
makeROC([ EmotionNB, EmotionXGB, EmotionRF, EmotionADA, EmotionSGD],"Sentiment, POS, and Entity Real Vs Fake Performance", Test_X_Emotion, Test_Y_Emotion)

# Feature Importance
# Load from file
pkl_filename = "EmotionADA.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionADA = pickle.load(file)

# Load from file
pkl_filename = "EmotionXGB.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionXGB = pickle.load(file)
    
# Load from file
pkl_filename = "EmotionRF.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionRF = pickle.load(file)

featurenames = EmotionXGB.get_booster().feature_names

pyplot.barh(range(len(EmotionADA.feature_importances_)), EmotionADA.feature_importances_)
plt.yticks(range(len(featurenames)), featurenames)
plt.title('ADABoost Feature Importance')
pyplot.show()

pyplot.barh(range(len(EmotionRF.feature_importances_)), EmotionRF.feature_importances_)
plt.yticks(range(len(featurenames)), featurenames)
plt.title('Random Forest Feature Importance')
pyplot.show()

pyplot.barh(range(len(EmotionXGB.feature_importances_)), EmotionXGB.feature_importances_)
plt.yticks(range(len(featurenames)), featurenames)
plt.title('XGBoost Feature Importance')
pyplot.show()

## Overall (All Features) Model

In [ ]:
dftopiccleaned = dftopic.drop(['neg'], axis = 1)
dfoverall = pd.concat([dftopiccleaned, dfemotion], axis=1)

#In case any columns are duplicated
#dfoverall = dfoverall.loc[:,~dfoverall.columns.duplicated()]

Train_X_overall, Test_X_overall, Train_Y_overall, Test_Y_overall = model_selection.train_test_split(dfoverall,save_y,train_size= 0.4, stratify=save_y, random_state=0)

print("*"*36)
print("SGD Optimization")
SGD = SGDClassifier(loss='log', penalty='l2', alpha=1e-3, n_iter_no_change=5)
param_dist = {
    'penalty': ['elasticnet','l2'],
    'alpha': [10 ** x for x in range(-6, 1)],
    'l1_ratio': [0, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1],
}
n_iter_search = 20
randomsearchmethod(SGD,param_dist, n_iter_search, Train_X_overall,Train_Y_overall)

print("*"*36)
print("ADABoost Optimization")
ADA = AdaBoostClassifier()
param_dist = {
    'learning_rate' : [0.05, 0.1, 0.2, 0.5],
    'n_estimators': [100,200,300,400],
    'base_estimator': [DecisionTreeClassifier(random_state=0, max_depth=2),DecisionTreeClassifier(random_state=0, max_depth=3)]
}
n_iter_search = 2
randomsearchmethod(ADA,param_dist, n_iter_search, Train_X_overall,Train_Y_overall)

#Restart this one it ran on emotion
print("*"*36)
print("Random Forest Optimization")
RF = RandomForestClassifier()
param_dist = {
    'max_depth': [30,35,40],
    'n_estimators': [110,120,130,140]
}
n_iter_search = 20
randomsearchmethod(RF,param_dist, n_iter_search, Train_X_overall,Train_Y_overall)

print("*"*36)
print("XGBoost Optimization")
XGB = XGBClassifier()
param_dist = {
    'max_depth': [15,20,25],
    'n_estimators': [ 150, 200, 250],
    'reg_alpha': [0.05, 0.1, 0.2, 0.5]
}
n_iter_search = 2
randomsearchmethod(XGB,param_dist, n_iter_search, Train_X_overall,Train_Y_overall)

# replace ngram_range = (1,2) for bigram
OverallNB = MultinomialNB().fit(Train_X_Topic,Train_Y_Topic)

OverallSGD = SGDClassifier(loss='log', penalty= 'elasticnet', l1_ratio= 0.1, alpha= 0.0001, n_iter_no_change=5).fit(Train_X_overall,Train_Y_overall)

OverallADA = AdaBoostClassifier(n_estimators= 300, learning_rate= 0.1, base_estimator= DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,)).fit(Train_X_overall,Train_Y_overall)

OverallRF = RandomForestClassifier(n_estimators=140, max_depth=30, random_state=0).fit(Train_X_overall,Train_Y_overall)

OverallXGB = XGBClassifier(n_estimators = 200, max_depth=25, reg_alpha=0.1).fit(Train_X_overall,Train_Y_overall)

# Save to file in the current working directory
pkl_filename = "OverallNB.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(OverallNB, file)

pkl_filename = "OverallSGD.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(OverallSGD, file)

pkl_filename = "OverallADA.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(OverallADA, file)

pkl_filename = "OverallRF.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(OverallRF, file)

pkl_filename = "OverallXGB.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(OverallXGB, file)

    # Load from file
pkl_filename = "OverallNB.pkl"
with open(pkl_filename, 'rb') as file:
    OverallNB = pickle.load(file)

# Load from file
pkl_filename = "OverallSGD.pkl"
with open(pkl_filename, 'rb') as file:
    OverallSGD = pickle.load(file)

# Load from file
pkl_filename = "OverallADA.pkl"
with open(pkl_filename, 'rb') as file:
    OverallADA = pickle.load(file)

# Load from file
pkl_filename = "OverallRF.pkl"
with open(pkl_filename, 'rb') as file:
    OverallRF = pickle.load(file)

# Load from file
pkl_filename = "OverallXGB.pkl"
with open(pkl_filename, 'rb') as file:
    OverallXGB = pickle.load(file)

predictions_NB = OverallNB.predict(Test_X_overall)
# Use accuracy_score function to get the accuracy
print("****NB Report****")
print("MultinomialNB Accuracy:",metrics.accuracy_score(Test_Y_overall, predictions_NB))
print('\nClasification report:\n', classification_report(Test_Y_overall, predictions_NB))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_overall, predictions_NB)  )
print()
print("*"*36)


# predict the labels on validation dataset
predictions_SGD = OverallSGD.predict(Test_X_overall)
# Use accuracy_score function to get the accuracy
print("****SGD Report****")
print("SGD Accuracy:",metrics.accuracy_score(Test_Y_overall, predictions_SGD))
print('\nClasification report:\n', classification_report(Test_Y_overall, predictions_SGD))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_overall, predictions_SGD))
print()
print("*"*36)


predictions_ADA = OverallADA.predict(Test_X_overall)
# Use accuracy_score function to get the accuracy
print("****ADABoost Report****")
print("AdaBoostClassifier Accuracy:",metrics.accuracy_score(Test_Y_overall, predictions_ADA))
print('\nClasification report:\n', classification_report(Test_Y_overall, predictions_ADA))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_overall, predictions_ADA) )
print()
print("*"*36)


predictions_RF = OverallRF.predict(Test_X_overall)
# Use accuracy_score function to get the accuracy
print("****RF Report****")
print("RandomForest Accuracy:",metrics.accuracy_score(Test_Y_overall, predictions_RF))
print('\nClasification report:\n', classification_report(Test_Y_overall, predictions_RF))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_overall, predictions_RF)  )
print()
print("*"*36)


predictions_XGB = OverallXGB.predict(Test_X_overall)
# Use accuracy_score function to get the accuracy
print("****XGB Report****")
print("XGBoost Accuracy:",metrics.accuracy_score(Test_Y_overall, predictions_XGB))
print('\nClasification report:\n', classification_report(Test_Y_overall, predictions_XGB))
print('\nConfussion matrix:\n',confusion_matrix(Test_Y_overall, predictions_XGB)  )

print()
print("*"*36)

makeROC([OverallNB, OverallXGB, OverallRF, OverallADA, OverallSGD],"TFIDF extract Real Vs Fake Performance", Test_X_overall, Test_Y_overall)

# Test on Entertainment Domain

## Combine data into a DataFrame

In [ ]:
SUBDIR = "entertainment"
FILENAME = ["real/", "fake/"]

file_directory = 'data/SampleText'
filename_pattern = '.+\.txt'

regex_pattern = "[a-zA-Z'_]+"

text = []
for file in FILENAME: 
    file_dir = DIR + SUBDIR + file
    entries = os.listdir(file_dir)
    for f in entries:
        with open(file_dir + f, 'r', encoding="utf8") as file: 
            lines = file.readlines()
            # cleanedLines = [[w for w in l if re.search(regex_pattern,w)] for l in lines]
            joinedLines = ' '.join(lines)
            joinedLines = joinedLines.strip("\n")
            text.append(joinedLines)

label  = [1 for i in range(50)] + [0 for i in range(50)] 
title  = ["Entertainment" for i in range(100)]
index  = [i for i in range(100)]
df = pd.DataFrame({"index":index, "title":title, "text":text, "label":label})

df.to_excel(DIR + SUBDIR +"entertainment_text.xlsx")

## Preprocessing

In [ ]:
otherdf = pd.read_excel('sample_data/entertainment_text.xlsx')
otherdf['text'] = otherdf.apply(lambda x: preprocessG(x['text']), axis=1)
otherdf.to_excel(DIR + SUBDIR + 'entertainment_text_clean.xlsx', index = False, header=True)
entertainmentdf = pd.read_excel(DIR + SUBDIR + 'entertainment_text_clean.xlsx')


## Topic Modelling

In [ ]:
news_e = entertainmentdf['text']

entertainment_docs = [[w for w in new.split()] for new in news ]
entertainment_vecs = docs2vecs(entertainment_docs, news_dictionary)

classifier_saved = open("topicModel14.pickle", "rb") #binary read
classifier_load = pickle.load(classifier_saved)
classifier_saved.close()
df_topic_sents_keywords_e = format_topics_sentences(ldamodel=classifier_load, corpus=entertainment_vecs, data=entertainment_docs)

# Format
# df_dominant_topic = df_topic_sents_keywords.reset_index()
df_topic_sents_keywords_e.columns = ['Topic 0','Topic 1','Topic 2','Topic 3','Topic 4','Topic 5', 'Topic 6','Topic 7','Topic 8','Topic 9','Topic 10','Topic 11','Topic 12','Topic 13','Text']

# Show
# print (df_topic_sents_keywords_e.head(10))
df_topic_sents_keywords_e.to_excel('entertainment_text_topic_dist.xlsx')


## POS Tag + Emotions/Sentiment Analysis

In [ ]:
entertainmentdf_emo = generate_emotions(entertainmentdf)
entertainmentdf_emo.drop_duplicates(inplace=True)
entertainmentdf_emo.to_excel(DIR + SUBDIR + 'emotion_analysis_final_entertainment.xlsx')

emp_df_e = pd.read_excel(DIR + SUBDIR + 'emotion_analysis_final_entertainment.xlsx')

emo_pos_df_e = generate_pos_tag_dist(emp_df_e)
emp_df_e = emp_df_e.reset_index()
emp_df_e = emp_df_e[emp_df_e.columns.drop(list(emp_df_e.filter(regex='index')))]
emp_df_e = emp_df_e.rename(columns={emp_df_e.columns[0]: 'index'})
comb_emo_pos_dist_e =  emp_df_e.merge(emo_pos_df_e, left_on='index', right_on='index')

emo_pos_dist_pscore_e = vader_score_generation(comb_emo_pos_dist_e)
emo_pos_dist_pscore_e.to_excel(DIR + SUBDIR + 'emo_pos_dist_pscore_entertainment.xlsx')

# Generating of Pos Grouping Features 
emotional_df_stack_e = pd.read_excel(DIR + SUBDIR + 'emo_pos_dist_pscore_entertainment.xlsx')
del emotional_df_stack_e['compound']

emotional_df_stack_groupped_pos_e, pos_tags_e = group_pos_features(emotional_df_stack_e)
emotional_df_stack_groupped_pos_e.drop(columns=pos_tags_e, inplace=True)
emotional_df_stack_groupped_pos_e = emotional_df_stack_groupped_pos_e.loc[:, (emotional_df_stack_groupped_pos_e != 0).any(axis=0)]
emotional_df_stack_groupped_pos_e.to_excel('sample_data/pos_groupings_vader_emotion.xlsx')


## Entity Recognition

In [ ]:

# Fake News
df1fake_e = otherdf[otherdf['label'] == 0]                                 # 1) Get fake news article dataframe
index_list_e = df1fake_e.index.tolist()                                 # 2) Get index of fake dataframe
# fakeEntityDic = getEntityDic(df1fake)                               # 3) Get sorted dict of {fake entities : count}
# fakeTop = getTopEntityDic(fakeEntityDic, 10)                        # 4) Get dic of {entitiy type : [sorted list of top 10 entities]}
df1_fakeEntities_e = addEntitiesToDataframe2(df1fake_e)                 # 5) Get DF with all entities, entity type count per article

# Real News
df1true_e = otherdf[otherdf['label'] == 1]                                 # 1) Get real news article dataframe
index_list_e = df1true_e.index.tolist()                                 # 2) Get index of real dataframe
# trueEntityDic = getEntityDic(df1true)                               # 3) Get sorted dict of {real entities : count}
# trueTop = getTopEntityDic(trueEntityDic, 10)                        # 4) Get dic of {entitiy type : [sorted list of top 10 entities]}
df1_trueEntities_e = addEntitiesToDataframe2(df1true_e)                 # 5) Get DF of entities, entity count per article

# Combining Real and Fake News Dataframes
finalEntitydf_e = pd.concat([df1_trueEntities_e, df1_fakeEntities_e])     # Union fake & real dataframes
finalEntitydf_e.sort_index(ascending=True, inplace=True)              # Re-index them to original state

# # Attaching Top Entities as Column Headers to Main Dataframe                   
# finalEntitydf_e, store_e = attachTopEntities(finalEntitydf_e, fakeTop_e, trueTop_e)       

# # Get Final DF with Top Entity counts
# mainEntitydf_e = getFinalEntityDF(finalEntitydf_e, store_e)
# mainEntitydf_e.head(3)

del finalEntitydf_e['person entities']
del finalEntitydf_e['organisation entities']
del finalEntitydf_e['location entities']
del emotional_df_stack_groupped_pos_e['Unnamed: 0']

horizontal_stack_e = emotional_df_stack_groupped_pos_e.merge(finalEntitydf_e, how='left', on='index')
horizontal_stack_e = horizontal_stack_e.loc[:,~horizontal_stack_e.columns.duplicated()]
del horizontal_stack_e['label']
del horizontal_stack_e['label_x']
del horizontal_stack_e['title_y']
del horizontal_stack_e['text_y']
del horizontal_stack_e[',']
del horizontal_stack_e['.']

horizontal_stack_e.to_excel('final_feature_engineered_entertainment.xlsx')

## Classification

In [ ]:
#create Sentiment dataset
dfemotionEntertainment = pd.read_excel("sample_data/Entertainment text + Sentiment Data.xlsx")
dfEntertainment = dfemotionEntertainment['text_x']
dfEntertainmentY = dfemotionEntertainment['label_y']
dfemotionEntertainment = dfemotionEntertainment.drop(['text_x','title_x','Unnamed: 0','index','label_y',':','(',')','group_c','PRP','NNP','group_m'], axis=1)

#Split text set to 20k rows from original extract
Train_X_Entertainment, Test_X_Entertainment, Train_Y_Entertainment, Test_Y_Entertainment = model_selection.train_test_split(df['text'],df['label'],train_size= 0.45, stratify=df['label'], random_state=0)

#TFIDF Vectorize New Sample Set with trained vectorizer from old domain extract
fulllist = []
for i in Train_X_Entertainment:
    fulllist.append(i)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(fulllist)
AlternateTrain_X_Tfidf = Tfidf_vect.transform(dfEntertainment)

#Convert to dataframe
Entertainment_Train_X_df = pd.DataFrame(AlternateTrain_X_Tfidf.toarray(), columns=Tfidf_vect.get_feature_names())

#Create Topic Entertainment DF
dftopicEntertainment = pd.read_excel("sample_data/topic_distribution_14_entertainment.xlsx")
dftopicEntertainment = dftopicEntertainment.drop(['Unnamed: 0','Text'], axis=1)
dftopicEntertainment = pd.concat([dftopicEntertainment, Entertainment_Train_X_df], axis=1)

#Create Overall Entertainment Dataset
Entertainmentdftopiccleaned = dftopicEntertainment.drop(['neg'], axis = 1)
dfoverallEntertainment = pd.concat([Entertainmentdftopiccleaned, dfemotionEntertainment], axis=1)

### Base Model

In [ ]:
# Load from file
pkl_filename = "BaseExtractTFIDFNB.pkl"
with open(pkl_filename, 'rb') as file:
    TFIDFNB = pickle.load(file)

# Load from file
pkl_filename = "BaseExtractTFIDFSGD.pkl"
with open(pkl_filename, 'rb') as file:
    TFIDFSGD = pickle.load(file)

# Load from file
pkl_filename = "BaseExtractTFIDFADA.pkl"
with open(pkl_filename, 'rb') as file:
    TFIDFADA = pickle.load(file)

# Load from file
pkl_filename = "BaseExtractTFIDFRF.pkl"
with open(pkl_filename, 'rb') as file:
    TFIDFRF = pickle.load(file)

# Load from file
pkl_filename = "BaseExtractTFIDFXGB.pkl"
with open(pkl_filename, 'rb') as file:
    TFIDFXGB = pickle.load(file)

    # replace ngram_range = (1,2) for bigram
predictions_NB = TFIDFNB.predict(Entertainment_Train_X_df)
# Use accuracy_score function to get the accuracy
print("****NB Report****")
print("MultinomialNB Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_NB))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_NB))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_NB)  )
print()
print("*"*36)


# predict the labels on validation dataset
predictions_SGD = TFIDFSGD.predict(Entertainment_Train_X_df)
# Use accuracy_score function to get the accuracy
print("****SGD Report****")
print("SGD Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_SGD))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_SGD))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_SGD))
print()
print("*"*36)


predictions_ADA = TFIDFADA.predict(Entertainment_Train_X_df)
# Use accuracy_score function to get the accuracy
print("****ADABoost Report****")
print("AdaBoostClassifier Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_ADA))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_ADA))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_ADA) )
print()
print("*"*36)


predictions_RF = TFIDFRF.predict(Entertainment_Train_X_df)
# Use accuracy_score function to get the accuracy
print("****RF Report****")
print("RandomForest Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_RF))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_RF))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_RF)  )
print()
print("*"*36)


predictions_XGB = TFIDFXGB.predict(Entertainment_Train_X_df)
# Use accuracy_score function to get the accuracy
print("****XGB Report****")
print("XGBoost Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_XGB))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_XGB))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_XGB)  )

print()
print("*"*36)

makeROC([TFIDFNB, TFIDFXGB, TFIDFRF, TFIDFADA, TFIDFSGD],"TFIDF extract Real Vs Fake Performance",Entertainment_Train_X_df, dfEntertainmentY)

### Base + Topic Model

In [ ]:
# Load from file
pkl_filename = "TopicTFIDFNB.pkl"
with open(pkl_filename, 'rb') as file:
    TopicNB = pickle.load(file)

# Load from file
pkl_filename = "TopicTFIDFSGD.pkl"
with open(pkl_filename, 'rb') as file:
    TopicSGD = pickle.load(file)

# Load from file
pkl_filename = "TopicTFIDFADA.pkl"
with open(pkl_filename, 'rb') as file:
    TopicADA = pickle.load(file)

# Load from file
pkl_filename = "TopicTFIDFRF.pkl"
with open(pkl_filename, 'rb') as file:
    TopicRF = pickle.load(file)

# Load from file
pkl_filename = "TopicTFIDFXGB.pkl"
with open(pkl_filename, 'rb') as file:
    TopicXGB = pickle.load(file)

# replace ngram_range = (1,2) for bigram
predictions_NB = TopicNB.predict(dftopicEntertainment)
# Use accuracy_score function to get the accuracy
print("****NB Report****")
print("MultinomialNB Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_NB))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_NB))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_NB)  )
print()
print("*"*36)

# predict the labels on validation dataset
predictions_SGD = TopicSGD.predict(dftopicEntertainment)
# Use accuracy_score function to get the accuracy
print("****SGD Report****")
print("SGD Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_SGD))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_SGD))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_SGD))
print()
print("*"*36)


predictions_ADA = TopicADA.predict(dftopicEntertainment)
# Use accuracy_score function to get the accuracy
print("****ADABoost Report****")
print("AdaBoostClassifier Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_ADA))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_ADA))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_ADA) )
print()
print("*"*36)


predictions_RF = TopicRF.predict(dftopicEntertainment)
# Use accuracy_score function to get the accuracy
print("****RF Report****")
print("RandomForest Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_RF))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_RF))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_RF)  )
print()
print("*"*36)


predictions_XGB = TopicXGB.predict(dftopicEntertainment)
# Use accuracy_score function to get the accuracy
print("****XGB Report****")
print("XGBoost Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_XGB))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_XGB))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_XGB)  )

makeROC([TopicNB, TopicXGB, TopicRF, TopicADA, TopicSGD],"TFIDF + Topic probabilities Real Vs Fake Performance",dftopicEntertainment, dfEntertainmentY)

### Generic Model

In [ ]:
# Load from file
pkl_filename = "EmotionNB.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionNB = pickle.load(file)

# Load from file
pkl_filename = "EmotionSGD.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionSGD = pickle.load(file)

# Load from file
pkl_filename = "EmotionADA.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionADA = pickle.load(file)

# Load from file
pkl_filename = "EmotionRF.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionRF = pickle.load(file)

# Load from file
pkl_filename = "EmotionXGB.pkl"
with open(pkl_filename, 'rb') as file:
    EmotionXGB = pickle.load(file)

# replace ngram_range = (1,2) for bigram
predictions_NB = EmotionNB.predict(dfemotionEntertainment)
# Use accuracy_score function to get the accuracy
print("****NB Report****")
print("MultinomialNB Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_NB))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_NB))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_NB)  )
print()
print("*"*36)


# predict the labels on validation dataset
predictions_SGD = EmotionSGD.predict(dfemotionEntertainment)
# Use accuracy_score function to get the accuracy
print("****SGD Report****")
print("SGD Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_SGD))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_SGD))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_SGD))
print()
print("*"*36)


predictions_ADA = EmotionADA.predict(dfemotionEntertainment)
# Use accuracy_score function to get the accuracy
print("****ADABoost Report****")
print("AdaBoostClassifier Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_ADA))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_ADA))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_ADA) )
print()
print("*"*36)


predictions_RF = EmotionRF.predict(dfemotionEntertainment)
# Use accuracy_score function to get the accuracy
print("****RF Report****")
print("RandomForest Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_RF))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_RF))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_RF)  )
print()
print("*"*36)


predictions_XGB = EmotionXGB.predict(dfemotionEntertainment)
# Use accuracy_score function to get the accuracy
print("****XGB Report****")
print("XGBoost Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_XGB))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_XGB))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_XGB)  )

makeROC([ EmotionNB, EmotionXGB, EmotionRF, EmotionADA, EmotionSGD],"Sentiment, POS, and Entity Real Vs Fake Performance", dfemotionEntertainment, dfEntertainmentY)


### Overall Model

In [ ]:
# Load from file
pkl_filename = "OverallNB.pkl"
with open(pkl_filename, 'rb') as file:
    OverallNB = pickle.load(file)

# Load from file
pkl_filename = "OverallSGD.pkl"
with open(pkl_filename, 'rb') as file:
    OverallSGD = pickle.load(file)

# Load from file
pkl_filename = "OverallADA.pkl"
with open(pkl_filename, 'rb') as file:
    OverallADA = pickle.load(file)

# Load from file
pkl_filename = "OverallRF.pkl"
with open(pkl_filename, 'rb') as file:
    OverallRF = pickle.load(file)

# Load from file
pkl_filename = "OverallXGB.pkl"
with open(pkl_filename, 'rb') as file:
    OverallXGB = pickle.load(file)

predictions_NB = OverallNB.predict(dfoverallEntertainment)
# Use accuracy_score function to get the accuracy
print("****NB Report****")
print("MultinomialNB Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_NB))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_NB))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_NB)  )
print()
print("*"*36)


# predict the labels on validation dataset
predictions_SGD = OverallSGD.predict(dfoverallEntertainment)
# Use accuracy_score function to get the accuracy
print("****SGD Report****")
print("SGD Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_SGD))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_SGD))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_SGD))
print()
print("*"*36)


predictions_ADA = OverallADA.predict(dfoverallEntertainment)
# Use accuracy_score function to get the accuracy
print("****ADABoost Report****")
print("AdaBoostClassifier Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_ADA))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_ADA))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_ADA) )
print()
print("*"*36)


predictions_RF = OverallRF.predict(dfoverallEntertainment)
# Use accuracy_score function to get the accuracy
print("****RF Report****")
print("RandomForest Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_RF))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_RF))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_RF)  )
print()
print("*"*36)


predictions_XGB = OverallXGB.predict(dfoverallEntertainment)
# Use accuracy_score function to get the accuracy
print("****XGB Report****")
print("XGBoost Accuracy:",metrics.accuracy_score(dfEntertainmentY, predictions_XGB))
print('\nClasification report:\n', classification_report(dfEntertainmentY, predictions_XGB))
print('\nConfussion matrix:\n',confusion_matrix(dfEntertainmentY, predictions_XGB)  )

print()
print("*"*36)

makeROC([OverallNB, OverallXGB, OverallRF, OverallADA, OverallSGD],"TFIDF extract Real Vs Fake Performance",dfoverallEntertainment, dfEntertainmentY)